<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Naas Chat Plugin

## Input

### Import libraries

In [ ]:
import os
import glob
import pydash
import json
import naas_python
import naas_data_product

## Model

### Create Chat commands from directory

In [ ]:
def create_command(input_dir):
    # Init
    commands = []
    connectors = []
    files = sorted(glob.glob(os.path.join(input_dir, "**", "*.ipynb"), recursive=True) + connectors, key=lambda x: x.split("/")[-1])

    # Loop on .ipynb files
    for file in files:
        # Init
        command = {}
        section = file.split("/")[-2]
        file_name = file.split("/")[-1]

        # Exclude file name chat plugin or pipeline
        if not file_name.startswith("__") and not file_name.endswith("__") and not "utils" in file and not '/home/ftp/.naas/' in file:
            # Open file and get cells
            with open(file) as f:
                nb = json.load(f)
            cells = nb.get("cells")

            # Find tags #command to generate command
            is_command = False
            for cell in cells:
                source = cell.get("source")
                if "#command" in " ".join(source):
                    is_command = True
                    break

            if is_command:
                # Get name & webhook
                payload = {}
                command["name"] = file_name.replace(".ipynb", "")
                command["avatar"] = cells[0].get("source")[0].split('src="')[1].split('"')[0]
                command["action"] =  {
                    "request_type": "POST",
                    "url": naas.webhook.add(file, params={"inline": True}),
                    "payload": {}
                }
                payload = {}

                # Add command description & payload
                for i, cell in enumerate(cells):
                    source = "".join(cell.get("source"))
                    tags = pydash.get(cell, "metadata.tags")
                    # Create description
                    if "**Description:**" in source:
                        command_description = source.split("**Description:**")[1].strip()
                        command["description"] = command_description.replace('notebook', 'command')
                    # Create command payload
                    if tags and "variables" in tags or "Setup variables" in source:
                        variables = source
                        defaults = "".join(cells[i+1].get("source"))
                    if tags and "parameters" in tags:
                        for s in cells[i+1].get("source"):
                            if "=" in s:
                                parameter = s.split("=")[0].strip()
                                payload[parameter] = {
                                    "type": "str",
                                    "description": variables.split(f"`{parameter}`:")[-1].split("-")[0].replace("\n", "").strip(),
                                    "default": defaults.split(f"{parameter} =")[-1].split("\n")[0].strip(),
                                }
                command["action"]["payload"] = payload

                # Generate commands (to be develop)
                commands.append(command)
#     print(json.dumps(commands, indent=4))
    return commands

# commands = create_command("/home/ftp/abi/models")
# commands

### Create plugin

In [ ]:
def create_plugin(
    name,
    prompt="",
    model="gpt-3.5-turbo-16k",
    temperature=0,
    output_dir=None,
    output_path=None,
    commands=[],
    description="",
    avatar="",
    prompt_type="system",
    entity_name=None
):
    """
    Creates a JSON file for a chat plugin with specified parameters and saves it to the specified output path.

    This function checks the number of tokens in the prompt, creates a JSON object, and saves it to a JSON file.
    It then creates an asset with the JSON file and returns the asset link.

    Parameters:
    - name (str): The name of the plugin.
    - prompt (str): The prompt for the plugin.
    - model (str): The name of the model to be used for tokenization. Default is "gpt-3.5-turbo-16k".
    - temperature (int): The temperature parameter for the model. Default is 0.
    - output_path (str): The path where the JSON file should be saved. If not provided, it will be created from the plugin name.
    - commands (list): Webhook command to be executed to be executed to get insert data into your Naas Chat.
    - description (str): Plugin description.
    - avatar (str): Link to PNG to be displayed as avatar in your Chat.
    - prompt_type (str): By default "system" but could be "assistant" or "human"

    Returns:
    str: The output path of the naas chat plugin.
    """
    # Init
    plugin = {}
    
    # Create output path
    if not output_path:
        output_path = remove_emojis(os.path.join(output_dir, f"{name.lower().replace(' ', '_')}.json"))
        
    # Create empty plugin
    if not os.path.exists(output_path):
        with open(output_path, "w") as f:
            json.dump(plugin, f)
        naas.asset.add(output_path, params={"inline": True})
        
    # Get asset URL
    plugin_url = naas.asset.find(output_path)
    
    # Check tokens
    prompt_tokens, max_tokens = naas_chat_plugin.check_tokens(prompt, model)

    # Create JSON
    entity_id = entity_name.lower().replace(' ', '-')
    plugin = {
        "id": f"{name.lower().replace(' ', '-')}-{entity_id}",
        "slug": f"{name.lower().replace(' ', '-')}/{entity_id}",
        "url": plugin_url,
        "name": f"{name} - {entity_name}",
        "model": model,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "prompt": prompt,
        "commands": commands,
        "description": description,
        "avatar": avatar,
        "prompt_type": prompt_type,
    }

    # Save dict to JSON file
    with open(output_path, "w") as f:
        json.dump(plugin, f)
    print(f"💾 Plugin successfully saved. You can use it in your Naas Chat with: {output_path}")
    naas.asset.add(output_path, params={"inline": True})
    return output_path, plugin_url, plugin

### Push plugin to Personal workspace

In [ ]:
def push_plugin_to_workspace(
    api_key,
    plugin,
    workspace_ids=[],
    personal_workspace=True
):
    # Init
    result = None
    if isinstance(workspace_ids, str) and workspace_ids == '':
        workspace_ids = []
        
    # Get workspaces
    workspaces = list_workspaces(api_key)
    
    # Get existing workspace ids
    current_workspace_ids = [workspace.get("id") for workspace in workspaces.get("workspaces")]

    if personal_workspace:
        personal_workspace_id = None
        # Get personal workspace
        for workspace in workspaces.get("workspaces"):
            if workspace.get("is_personal"):
                personal_workspace_id = workspace.get("id")
                break
        if personal_workspace_id not in workspace_ids:
            workspace_ids.append(personal_workspace_id)
    
    for index, workspace_id in enumerate(workspace_ids):
        if workspace_id in current_workspace_ids:
            workspace_id = workspace_id.strip()
            print(f"{index} - Workspace ID:", workspace_id)
            # List existing plugins
            plugins = list_workspace_plugins(
                api_key,
                workspace_id,
            )
            plugin_exist = False
            print("Plugins:", len(plugins.get('workspace_plugins')))
            for i, p in enumerate(plugins.get('workspace_plugins')):
                plugin_id = p.get("id")
                p_json = json.loads(p.get("payload"))
                p_name = p_json.get("name")
                p_slug = p_json.get("slug")
                p_url = p_json.get("url")
                if plugin_url == p_url:
                    plugin_exist = True
                    print("Plugin ID:", plugin_id)
                    print("- Slug:", p_slug)
                    print("- Name:", p_name)
                    print("- URL:", p_url)
                    break

            # Create or update plugin
            if not plugin_exist:
                result = create_workspace_plugin(
                    api_key,
                    workspace_id,
                    plugin,
                )
                print("✅ Plugin successfully created")
            else:
                result = update_workspace_plugin(
                    api_key,
                    workspace_id,
                    plugin_id,
                    plugin,
                )
                print("✅ Plugin successfully updated")
            print()
        else:
            print("❌ Workspace ID does not exist for user.")
    return result

## Output